In [1]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:

batch_size = 128
epochs = 1

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


422/422 [==============================] - 81s 190ms/step - loss: 0.7579 - accuracy: 0.7640 - val_loss: 0.0934 - val_accuracy: 0.9763


In [6]:
test = model.predict(x_test)

In [5]:
tf.saved_model.save(model, 'models/1/')

INFO:tensorflow:Assets written to: models/1/assets


In [7]:
import requests
import stackn.auth as sa

In [8]:
inp = {"inputs": x_test[0:10].tolist()}

In [28]:
conf, status = sa.get_token()
res = requests.post('https://tensorflow-serving-mnist-test-hfa-9045.studio.local.stackn.dev/v1/models/models:predict', json=inp, headers={"Authorization": "Bearer "+conf["STACKN_ACCESS_TOKEN"]})

In [29]:
res.json()

{'outputs': [[1.65731304e-08,
   1.23818367e-09,
   9.12531618e-07,
   3.76514663e-06,
   9.88239351e-11,
   2.95853275e-09,
   5.86294088e-13,
   0.999992728,
   9.22080101e-09,
   2.48703441e-06],
  [0.00324816559,
   0.00168007705,
   0.993385732,
   5.16019682e-05,
   2.36656561e-08,
   2.24792e-06,
   0.00151112874,
   4.37433423e-09,
   0.000121032703,
   3.04132497e-10],
  [1.10937117e-05,
   0.998290479,
   6.64464606e-05,
   1.7213817e-05,
   0.000734296453,
   1.55083558e-06,
   0.000128720058,
   0.000645991124,
   9.12914256e-05,
   1.29455311e-05],
  [0.999680281,
   6.85754509e-10,
   1.78727714e-05,
   1.03551379e-07,
   3.27164287e-07,
   2.28460749e-06,
   0.000256543426,
   1.5536456e-05,
   8.25512359e-07,
   2.61865753e-05],
  [7.80159098e-07,
   4.15953316e-08,
   4.07271529e-07,
   5.41890088e-08,
   0.999595225,
   4.25559143e-08,
   2.87392504e-06,
   1.960384e-05,
   6.81592098e-08,
   0.000380776764],
  [4.02169508e-06,
   0.998417258,
   2.28037588e-05,
   8.

In [27]:
conf["STACKN_TOKEN"]

'..._ACddSxV8gpj7OE4PSzVuMA8S8Hj0h_zY_RuMgzlasyWVLwTT6xGhjxeNHkxUam5XV6Vfqt6IT5KdgOPPlYnrOIsKXb4krVUG9Kr6T2Fask3TvhyJKCxthh8TTR1fkRFaiVNo4NSekw4IyxoDlCgY1z3Jmvrr6-ymk_i3kFMOsEIVJHdTw'